In [1]:
# Beginning of "supply_chain_example.ipynb"

# Supply Chain Example Using PuLP

A notebook that starts with Pandas dataframe and using the PuLP library to optimize this supply chain data.

In [2]:
# Import necessary libraries and modules
import pandas as pd
# import yaml
import os
from pulp import *
# import time

In [3]:
# Get the directory of the current workign directory
print(os.getcwd())

/run/media/joebravo/Extreme SSD/supply_chain_project/notebooks


In [4]:
df = pd.DataFrame(
    columns=[
        'LaneId',
        'Distance',
        'ImpactPer',
        'MinVol',
        'MaxVol'
    ]
)
df.loc[0] = [0, 570, 349.50, 2, 6]
df.loc[1] = [1, 117, 109.42, 0, 3]
df.loc[2] = [2, 120, 0.45, 15, 25]
df.loc[3] = [3, 1116, -915.59, 0, 0]
df.loc[4] = [4, 1058, -1000.56, 0, 0]
df.loc[5] = [5, 942, -550.68, 0, 0]
df.loc[6] = [6, 1103, -960.45, 0, 0]
df.loc[7] = [7, 921, -787.00, 0, 2]
df.loc[8] = [8, 1091, -927.75, 0, 0]
df.loc[9] = [9, 443, 82.08, 3, 8]
df.loc[10] = [10, 1688, -889.49, 0, 0]
df.loc[11] = [11, 432, -265.28, 2, 4]
df.loc[12] = [12, 451, -81.15, 8, 10]
df.loc[13] = [13, 505, -545.46, 0, 0]
df.loc[14] = [14, 444, -50.00, 0, 0]
df.loc[15] = [15, 1247, -878.54, 0, 0]
df.loc[16] = [16, 710, -837.95, 0, 2,]
df.loc[17] = [17, 323, -379.91, 0, 1,]
df.loc[18] = [18, 1035, -508.73, 1, 2]
df.loc[19] = [19, 1139, -946.08, 0, 0]
df.loc[20] = [20, 1136, -535.74, 0, 1]
df.loc[21] = [21, 1251, 512.79, 6, 6]
df.loc[22] = [22, 1107, -952.41, 0, 0]
df.loc[23] = [23, 1180, -1045.29, 0, 4]
df.loc[24] = [24, 268, 107.72, 4, 4]
df.loc[25] = [25, 863, -47.50, 0, 1]

df

,LaneId,Distance,ImpactPer,MinVol,MaxVol
0,0.0,570.0,349.50,2.0,6.0
1,1.0,117.0,109.42,0.0,3.0
2,2.0,120.0,0.45,15.0,25.0
3,3.0,1116.0,-915.59,0.0,0.0
4,4.0,1058.0,-1000.56,0.0,0.0
5,5.0,942.0,-550.68,0.0,0.0
6,6.0,1103.0,-960.45,0.0,0.0
7,7.0,921.0,-787.00,0.0,2.0
8,8.0,1091.0,-927.75,0.0,0.0
9,9.0,443.0,82.08,3.0,8.0


In [5]:
"""Set High Level Constraints."""
OPTSET_MILE_LIMIT_UPPER = 22000
OPTSET_MILE_LIMIT_LOWER = 20000

In [6]:
"""Init lp solver."""
prob = pulp.LpProblem('LaneSelectionOptimization', LpMaximize)

"""Solver setup."""
Lanes = df.index
MaxVols = df['MaxVol']
MinVols = df['MinVol']
Impacts = df['ImpactPer']
Miles = df['Distance']

In [7]:
x = LpVariable.dicts('Lane', Lanes, None, None, LpInteger)

for l in Lanes:
    x[l].bounds(MinVols[l], MaxVols[l])

"""Set the Objective Funtion."""
prob += sum([x[l] * Impacts[l] for l in Lanes]), 'Sum_of_Impact'

"""Set the Constraints."""
prob += lpSum([Miles[l] * x[l] for l in Lanes]) <= OPTSET_MILE_LIMIT_UPPER
prob += lpSum([Miles[l] * x[l] for l in Lanes]) >= OPTSET_MILE_LIMIT_LOWER

In [8]:
"""Wrap up and Solve."""
LpSolverDefault.msg = 1
prob.writeLP('LaneOpt.lp')
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/joebravo/.venv/supply_chain/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/fc3fe79efe5e4eb7bdf825a48de939ca-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/fc3fe79efe5e4eb7bdf825a48de939ca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 138 RHS
At line 141 BOUNDS
At line 174 ENDATA
Problem MODEL has 2 rows, 26 columns and 52 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4685.44 - 0.00 seconds
Cgl0004I processed model has 1 rows, 13 columns (13 integer (3 of which binary)) and 13 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0012I Integer solution of -4661.99 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0038I Full problem 1 rows 13 columns, reduced to 0 rows 0 columns

1

In [9]:
"""Handle results."""
TotalImpact = prob.objective.value()
Status = LpStatus[prob.status]
print('Status:', Status)
print('Total Impact:', TotalImpact)

flow = {l:x[l].varValue for l in Lanes}

output = []
for lane in Lanes:
    var_output = {
        'Lane': lane,
        'Production': flow[lane]
    }
    output.append(var_output)

dfOptResults = pd.DataFrame.from_records(output)
dfOptResults.set_index('Lane', inplace=True)

Status: Optimal
Total Impact: 4661.99


In [10]:
"""Merge results with orginal data."""
data = pd.merge(df, dfOptResults, how='left', left_index=True, right_index=True)

In [11]:
"""Bonus: Calc impact for each lane."""
data['LaneImpact'] = data['ImpactPer'] * data['Production']

In [12]:
# End of "supply_chain_example.ipynb"